**Custom Tool:**

A custom tool is a tool that you define yourself with your specific functionality.

**When to use:**

--> You want to call you own API.

--> You want to encapsulate bussiness logic.

--> You want the LLM to interact with your database, product or app.

# **Method 1 to Create Custom Tools : Using @tool decorator**

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
from langchain_core.tools import tool

In [4]:
# STEP 1 : Create a Function with doc string (string that higlights what the function/tool is performing)

def multiply(a,b):
  """Multiply two integers"""
  return a*b

In [6]:
# STEP 2 : Add type hints

def multiply(a:int, b:int) -> int:
  """Multiply two integers"""
  return a*b

In [7]:
# STEP 3 : Add Tool Decorator

@tool
def multiply(a:int, b:int) -> int:
  """Multiply two integers"""
  return a*b

In [9]:
# ✅ Note:
# We pass {"a": 10, "b": 3} as a dictionary because @tool converts the function into a LangChain tool that expects JSON-style input, just like an LLM would send.
# The keys match parameter names, and LangChain auto-converts values (e.g., "10" → 10) based on type hints.

result = multiply.invoke({"a":10,"b":3})

In [10]:
print(result)

30


In [11]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [13]:
# What LLM see when we pass this tool to LLM
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two integers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# **Method 2 to Create Custom Tool : Using StructuredTool and Pydantic**

A StructuredTool in Langchain is a special type of tool where the input to the tool follows a structured schema, typically defined using the Pydantic Model

In [15]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [16]:
class MultiplyBase(BaseModel):
  a:int = Field(required=True, description="First number")
  b:int = Field(required=True, description="Second number")

In [17]:
def Multiply(a:int, b:int) -> int:
  """Multiply two integers"""
  return a*b

In [18]:
multiply = StructuredTool.from_function(
    func=Multiply,
    name="Multiply",
    description="Multiply two integers",
    args_schema=MultiplyBase
)

In [21]:
result2 = multiply.invoke({"a":11,"b":3})

In [22]:
print(result2)

33


In [23]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

Multiply
Multiply two integers
{'a': {'description': 'First number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number', 'required': True, 'title': 'B', 'type': 'integer'}}


# **Method 3 to Create Custom Tool : Using BaseModel Class**

BaseTool is the abstract base class for all tools in LangChain.
It defines the core structure and interface that any tool must follow, whether it's a simple one-liner or a fully customized function.

All other tool types like @tool, StructuredTool are built on top of BaseTool.

In [37]:
from langchain_core.tools import StructuredTool
from typing import Type

In [38]:
class MultiplyBase(BaseModel):
  a:int = Field(required=True, description="First number")
  b:int = Field(required=True, description="Second number")

In [42]:
class MultiplyTool(StructuredTool):
  name:str = "Multiply"
  description:str = "Multiply two integers"

  args_schema: Type[BaseModel] = MultiplyBase

  def _run(self, a:int, b:int) -> int:
    return a*b

In [43]:
multiply_tool = MultiplyTool()

In [44]:
result3 = multiply_tool.invoke({"a":12,"b":3})

In [45]:
print(result3)

36


# **Toolkit:**

In LangChain, a Toolkit is simply a collection of related tools grouped together to perform a specific type of task.

Each Tool is like a small function (e.g., search, read file, send email), and a Toolkit bundles several such tools that serve a common purpose.

In [47]:
@tool
def add(a:int, b:int)-> int:
  "Add two numbers"
  return a+b

In [48]:
@tool
def subtract(a:int,b:int)->int:
  "Subtract two numbers"
  return a-b

In [49]:
# Make a Toolkit which combines all the tools
class MathToolkit:
  def get_tools(self):
    return [add,subtract]

In [50]:
# Access Tools using creating the object of Toolkit Function
toolkit = MathToolkit()

In [52]:
# Get all available tools in toolkit
tools = toolkit.get_tools()

for i in tools:
  print(i.name,"==>",i.description)

add ==> Add two numbers
subtract ==> Subtract two numbers


In [53]:
# Can use this tools:
r1 = add.invoke({"a":10,"b":3})
r2 = subtract.invoke({"a":10,"b":3})

print(r1)
print(r2)

13
7
